### Bibliotecas

In [1]:
# Matematicos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# GetDate
from datetime import datetime #,  date

### Importacao de listagem

In [2]:
#Reimportando arquivo - caso execucao tenha sido pausada
path_to_file = '..//ListagemExp/ListagemExport2016-2022_checkpoint_13-11-2022_00-50-47.csv'
listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

### Requisicao a Camara

In [4]:
# Preenchimento do campo Id_Projeto_Camara
count = 1
failed_req= 0
failed = False
id_projeto_camara = ''
max_failed_req = 3
notfound_count = 0
req_count = 0
max_req = 3500
export = False

while count > 0:
    for i in listagem.index:
        if (listagem['Id_Projeto_Camara'][i] == '' or listagem['Id_Projeto_Camara'][i] == 0) and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            with requests.Session() as session:
                id_projeto_camara = ''
                failed = False
                # Variaveis de requisicao
                num_proj = int(listagem['Projeto'][i])
                ano_proj = int(listagem['Ano'][i])
                tipo_proj = listagem['Codigo_Tipo'][i]

                try:
                    # Requisicao para o portal de propostas da Camara
                    query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                    #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                    resposta = session.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                    req_count+=1
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        #retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', '').replace('\n', '').replace('\r', '').replace('\t', ''))
                        id_projeto_camara = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        try:
                            retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                            id_projeto_camara = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                        except:
                            id_projeto_camara = 'Nao Encontrado'
                            notfound_count += 1
                except:
                    #Assinala que houve falha
                    failed = True
                    time.sleep(180)

                listagem['Id_Projeto_Camara'][i] = id_projeto_camara
                #resposta.close()
                session.close()
                time.sleep(0.5)

        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve X falhas consecutivas ou houve mais de Y requisicoes consecutivas 
        if failed_req > max_failed_req or req_count > max_req:
            break

    if failed_req > max_failed_req or req_count > max_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto_Camara'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1

now =datetime.now()
dthr_finalizado = now.strftime("%d-%m-%Y_%H-%M-%S")

try:
    # Exportacao da listagem no estado atual para arquivo csv
    file = 'ListagemExport2016-2012_checkpoint_' + dthr_finalizado
    path_to_file = '../ListagemExp/'+file+'.csv'
    path = Path(path_to_file)

    exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
                    'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

    if path.is_file():
        exp_header = False
    else:    
        exp_header = True

    listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

    export = True

except:
    export = False


if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
elif req_count > max_req:
    print(f'Processo finalizado em {req_count} requisicoes. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}. Finalizado em: {dthr_finalizado}. Status da exportacao: {export}.')


O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = 0. Finalizado em: 13-11-2022_06-40-45. Status da exportacao: True.


### Exportacao da listagem no estado atual 

In [ ]:
# Exportacao da listagem no estado atual para arquivo csv
now =datetime.now()

file = 'ListagemExport2012-2015_checkpoint_' + now.strftime("%d-%m-%Y_%H-%M-%S")
path_to_file = '../ListagemExp/'+file+'.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Autor','Codigo_Tipo', 
               'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']

if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 
print(path)

### Identificacao do estado da importacao dos dados

In [6]:
tem=0
naovaiter=0
naotem=0

for i in listagem.index:
    if listagem['Id_Projeto_Camara'][i] != 0 and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
    #if listagem['Id_Projeto_Camara'][i]!='' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
        tem+=1
    elif listagem['Codigo_Tipo'][i] in ('DESCONHECIDO','EMENDA'):
        naovaiter+=1
    else:
        naotem+=1

print(tem)
print(naotem)
print(naovaiter)

41548
20176
424


In [ ]:
display(listagem)